In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from models import *
from training_utils import *

In [2]:
import tensorflow as tf

In [4]:
#tf.random.set_seed(42)
np.random.seed(42)

In [5]:
import numpy as np

In [6]:
labels = np.load(file="../data/labels.npy")

In [7]:
plain_features = np.load(file="../data/plain_features.npy")
verb_embed = np.load(file="../data/elmo_verbs.npy")
arg_embed = np.load(file="../data/elmo_args.npy")

In [8]:
print("Shapes")
print(f"Labels: {labels.shape}")
print(f"Plain: {plain_features.shape}")
print(f"Verbs: {verb_embed.shape}")
print(f"Args: {arg_embed.shape}")

Shapes
Labels: (52751, 44)
Plain: (52751, 932)
Verbs: (52751, 1024)
Args: (52751, 1024)


In [11]:
simple_factory = SimpleModelFactory(932, number_of_roles=44)
concat_factory = SimpleModelFactory(932+1024+1024, number_of_roles=44)
sparse_factory = SparseModelFactory([932, 1024, 1024], number_of_roles=44)

## Simple model

In [ ]:
cv_res_simple = custom_cross_val(simple_factory.create_and_compile, [plain_features], labels, cv=DEFAULT_CV, epochs=13, batch_size=64, validation_split = 0., shuffle=True, verbose = 0)
describe_cv_result(cv_res_simple)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 600)               559800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 600)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 600)               2400      
_________________________________________________________________
dense_7 (Dense)              (None, 400)               240400    
_________________________________________________________________
batch_normalization_7 (Batch (None, 400)               1600      
_________________________________________________________________
activation_4 (Activation)    (None, 400)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 400)               0         
__________

In [13]:
describe_cv_result(cv_res_simple)

[[0.81473612 0.76182352 0.76182352 0.72614624 0.76182352]
 [0.82082522 0.76815166 0.76815166 0.72982825 0.76815166]
 [0.78318418 0.77317536 0.77317536 0.73670029 0.77317536]
 [0.78552898 0.77260664 0.77260664 0.73761419 0.77260664]
 [0.80872386 0.76654028 0.76654028 0.73277883 0.76654028]]
Mean
     loss  keras_accur  micro_f1  macro_f1     accur
0  0.8026     0.768459  0.768459  0.732614  0.768459
Std
       loss  keras_accur  micro_f1  macro_f1     accur
0  0.015397     0.004177  0.004177  0.004274  0.004177


In [14]:
chunk = np.concatenate((verb_embed, arg_embed, plain_features), axis=1)

# Simple Concatenated

In [15]:
cv_res_concat = custom_cross_val(concat_factory.create_and_compile, [chunk], labels, cv=DEFAULT_CV, epochs=13, batch_size=64, validation_split = 0., shuffle=True, verbose = 0)
describe_cv_result(cv_res_concat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 600)               1788600   
_________________________________________________________________
dropout_16 (Dropout)         (None, 600)               0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 600)               2400      
_________________________________________________________________
dense_25 (Dense)             (None, 400)               240400    
_________________________________________________________________
batch_normalization_25 (Batc (None, 400)               1600      
_________________________________________________________________
activation_16 (Activation)   (None, 400)               0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 400)               0         
__________

## Sparse Model

In [16]:
cv_res_sparse = custom_cross_val(sparse_factory.create_and_compile, [plain_features, verb_embed, arg_embed], labels, cv=DEFAULT_CV, epochs=13, batch_size=300, validation_split = 0., shuffle=True, verbose = 0)
describe_cv_result(cv_res_sparse)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input-predicate-embeddings (Inp (None, 1024)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
input-categorical (InputLayer)  (None, 932)          0                                            
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 100)          102500      input-predicate-embeddings[0][0] 
__________________________________________________________________________________________________
dense_44 (